In [ ]:
#MERGE DATASETS

In [ ]:
#Transformations needed for models (PLANT 1)
#1. Normalize or Scale Data: Use MinMaxScaler or StandardScaler to ensure all features are on the same scale for consistent model inputs, especially if you're merging datasets from multiple plants. 
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[['DC_POWER', 'AC_POWER', 'DAILY_YIELD', 'TOTAL_YIELD']] = scaler.fit_transform(df[['DC_POWER', 'AC_POWER', 'DAILY_YIELD', 'TOTAL_YIELD']])

In [ ]:
#2. Handle Skewness: Apply logarithmic transformations for features like DC_POWER and AC_POWER to reduce the impact of skewed distributions.
import numpy as np
df['LOG_DC_POWER'] = np.log1p(df['DC_POWER'])  # Avoid log(0) errors
df['LOG_AC_POWER'] = np.log1p(df['AC_POWER'])

In [ ]:
#3. Outlier Detection: For DAILY_YIELD and TOTAL_YIELD, identify and handle outliers. You could cap extreme values or remove rows beyond specific quantiles
# Cap values
q_low = df['DAILY_YIELD'].quantile(0.01)
q_high = df['DAILY_YIELD'].quantile(0.99)
df['DAILY_YIELD'] = np.clip(df['DAILY_YIELD'], q_low, q_high)

In [ ]:
# PLANT 1 WEATHER 

df['LOG_IRRADIATION'] = np.log1p(df['IRRADIATION'])  # log1p to handle zeros


In [ ]:
#PLANT 1 WEAHTER 
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
weather_features = ['AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE', 'IRRADIATION']
df[weather_features] = scaler.fit_transform(df[weather_features])